# Linear Graph with 16 nodes



***Quantum Walk***

Initial installation and authentications

In [3]:
!pip install -U classiq
import classiq
classiq.authenticate()

/usr/local/lib/python3.10/dist-packages/classiq/_internals/authentication/token_manager.py:99: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [31]:
from classiq import *

In [32]:
size = 4
# As there are 16 nodes, size has been chosen to be 4.

-- The following section remains the same as that of the circular graph quantum walk explained in the video which implements the phase kickback for states other than the 0 state.

---




In [33]:
@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)

@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)



@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)



# W Operator

1. This function creates the superposition of all the adjacent vertices for all vertices in the linear graph.

2. In this case, the first and the last has only one adjacent verex compared to all other vertices.


3. For all other vertices, the states are populated using the for loop as was explained in the video.

4. For 0th and 15th vertices, the for loop is customized accordingly.

In [34]:

def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    prob = [0 for  _ in range(2**size)]
    if i == 0:
      prob[i+1] = 1
    elif i == 15:
      prob[i-1] = 1
    else:
      prob[(i+1)% 2**size]=0.5
      prob[(i-1)% 2**size]=0.5
    print(f'State={i}, prob vec ={prob}')

    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))


@qfunc
def W_operator(vertices:QNum, adjacent_vertices: QNum):


    for i in range(2**size):
      print('i == ', i)
      W_iteration(i,vertices,adjacent_vertices)




# S Operator

1. The edge oracle is changed from the example.
2. In this case, the qubit should result in a 1 state for the edges of two adjacent vertices.
3. The code for edge oracle is changed accordingly.

In [35]:
from classiq.qmod.symbolic import logical_or

@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  #res |= (((vertices+adjacent_vertices)%2) ==1)
  res |= logical_or((adjacent_vertices == vertices + 1), (vertices == adjacent_vertices + 1) )


@qfunc
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))


@qfunc
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))



# MAIN FUNCTION

In [36]:
@qfunc
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):
  print(size)
  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

4
i ==  0
State=0, prob vec =[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i ==  1
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i ==  2
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i ==  3
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i ==  4
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i ==  5
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i ==  6
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
i ==  7
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
i ==  8
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
i ==  9
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
i ==  10
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
i ==  11
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
i ==  12
State=12, prob vec =[0, 0, 0,

In [11]:
from IPython.display import Image 
Image(url="quantum_walk_linear.jpg", width=900, height=300) 

